- QUESTÃO 2
    Infelizmente meu google drive está estourado de armazenamento e não consigo limpar a tempo. Dessa forma, irei somente deixar o codigo que eu usaria.


In [ ]:
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery    import build
from google.oauth2                import service_account
from google.cloud                 import bigquery
import requests
import gspread
import pandas                     as pd

#configurações para conexão ao GCP e suas ferramentas
SERVICE_ACCOUNT_FILE = '' #a gente teria aqui um arquivo de conta de serviço, que pode ser extraído pelo IAM do GCP. É também possivel usar um texto (copiando e colando) e transformar ele em arquivo, pois essa variavel funciona apenas se for um path de um arquivo.
SCOPES = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/drive']

bigquery_client = bigquery.Client()
query           = "SELECT login_md5, gender, ddd_estado FROM `project-iury.greenpeace.random_user`"
df              = client.query(query).to_dataframe()

pkmn         = ['bulbasaur', 'charmander', 'squirtle', 'chikorita', 'cyndaquil', 'totodile', 'treecko', 'torchic', 'mudkip', 'turtwig', 'chimchar', 'piplup', 'snivy', 'tepig', 'oshawott', 'chespin', 'fennekin', 'froakie', 'rowlet', 'litten', 'popplio', 'grookey', 'scorbunny', 'sobble']
pkmn_inicial = []

for pokemon in pkmn:
    url = f"https://pokeapi.co/api/v2/pokemon/{pokemon}"
    response = requests.get(url)
    data = response.json()
    pkmn_inicial.append([pokemon,data['types'][0]['type']['name']])

print(pkmn_inicial)

for index, row in df.iterrows():
    pkmn = random.choice(pkmn_inicial)
    df.loc[index, 'pkmn'] = pkmn[0]
    df.loc[index, 'tipo'] = pkmn[1]

-- CONECTANDO AO SHEETS 
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES) 
service     = build('drive', 'v3', credentials=credentials)

folder_water = {
    'name': 'water',
    'mimeType': 'application/vnd.google-apps.folder'
}

folder_grass = {
    'name': 'grass',
    'mimeType': 'application/vnd.google-apps.folder'
}

folder_fire = {
    'name': 'grass',
    'mimeType': 'application/vnd.google-apps.folder'
}

folder_water = service.files().create(body=folder_water, fields='id').execute()
folder_grass = service.files().create(body=folder_grass, fields='id').execute()
folder_fire = service.files().create(body=folder_fire, fields='id').execute()

id_folder_water = folder_water.get('id'))
id_folder_grass = folder_grass.get('id'))
id_folder_fire = folder_fire.get('id'))

file_metadata_grass_men = {
    'name': 'men.csv',
    'parents': id_folder_grass
}

media_grass_men = MediaFileUpload(df[(df['gender'] == 'male') & (df['tipo'] == 'grass')].to_csv('men.csv', index=False), mimetype='text/csv')
file_grass_men = service.files().create(body=file_metadata_grass_men, media_body=media_grass_men, fields='id').execute()
print('sucesso')


-- O mesmo para os demais (female) e também water e fire.from googleapiclient.discovery import build
from google.oauth2             import service_account
from google.cloud              import bigquery

import pandas     as pd
import requests

bigquery_client = bigquery.Client()
query           = "SELECT login_md5, gender, location_state, ddd_estado FROM `project-iury.greenpeace.random_user`"
df              = client.query(query).to_dataframe()

pkmn         = ['bulbasaur', 'charmander', 'squirtle', 'chikorita', 'cyndaquil', 'totodile', 'treecko', 'torchic', 'mudkip', 'turtwig', 'chimchar', 'piplup', 'snivy', 'tepig', 'oshawott', 'chespin', 'fennekin', 'froakie', 'rowlet', 'litten', 'popplio', 'grookey', 'scorbunny', 'sobble']
pkmn_inicial = []

for pokemon in pkmn:
    url = f"https://pokeapi.co/api/v2/pokemon/{pokemon}"
    response = requests.get(url)
    data = response.json()
    pkmn_inicial.append([pokemon,data['types'][0]['type']['name']])

print(pkmn_inicial)

for index, row in df.iterrows():
    pkmn = random.choice(pkmn_inicial)
    df.loc[index, 'pkmn'] = pkmn[0]
    df.loc[index, 'tipo'] = pkmn[1]

-- CONECTANDO AO SHEETS 
SERVICE_ACCOUNT_FILE = ''
SCOPES = ['https://www.googleapis.com/auth/drive']

credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service     = build('drive', 'v3', credentials=credentials)

folder_water = {
    'name': 'water',
    'mimeType': 'application/vnd.google-apps.folder'
}

folder_grass = {
    'name': 'grass',
    'mimeType': 'application/vnd.google-apps.folder'
}

folder_fire = {
    'name': 'grass',
    'mimeType': 'application/vnd.google-apps.folder'
}

folder_water = service.files().create(body=folder_water, fields='id').execute()
folder_grass = service.files().create(body=folder_grass, fields='id').execute()
folder_fire = service.files().create(body=folder_fire, fields='id').execute()

id_folder_water = folder_water.get('id'))
id_folder_grass = folder_grass.get('id'))
id_folder_fire = folder_fire.get('id'))

file_metadata_grass_men = {
    'name': 'men.csv',
    'parents': id_folder_grass
}

media_grass_men = MediaFileUpload(df[(df['gender'] == 'male') & (df['tipo'] == 'grass')].to_csv('men.csv', index=False), mimetype='text/csv')
file_grass_men = service.files().create(body=file_metadata_grass_men, media_body=media_grass_men, fields='id').execute()
print('sucesso')


-- O mesmo para os demais (female) e também water e fire.
-- ...

-- CRIANDO SHEETS com tipos por estado
df_state_type = df.groupby('location_state')['tipo'].nunique().reset_index()

gsheet_client = gspread.authorize(creds)
spreadsheet   = gsheet_client.create('Pokemon Type by State')
sheet         = spreadsheet.get_worksheet(0)
sheet.update([df.columns.values.tolist()] + df.values.tolist())
print("sucesso")